In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
print(torch.cuda.get_device_name(0) if device.type == "cuda" else "Apple Neural Engine")

class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.75)
        self.dropout2 = nn.Dropout(0.75)
        self.fc1 = nn.Linear(12544, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        #print(x.size())
        x = torch.flatten(x, 1)
        #print(x.size())
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

model = MNIST_CNN().to(device)
print(model)

Using device: cpu
Apple Neural Engine
MNIST_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout1): Dropout(p=0.75, inplace=False)
  (dropout2): Dropout(p=0.75, inplace=False)
  (fc1): Linear(in_features=12544, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        
        # Compute the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1 )*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size: >5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n -------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 -------------------------
loss: 2.300680 [   64/60000]
loss: 0.588638 [ 6464/60000]
loss: 0.315839 [12864/60000]
loss: 0.327128 [19264/60000]
loss: 0.073848 [25664/60000]
loss: 0.223440 [32064/60000]
loss: 0.156897 [38464/60000]
loss: 0.293132 [44864/60000]
loss: 0.309811 [51264/60000]
loss: 0.233633 [57664/60000]
Test Error:
 Accuracy: 98.0%, Avg loss: 0.060058

Epoch 2
 -------------------------
loss: 0.083324 [   64/60000]
loss: 0.181408 [ 6464/60000]
loss: 0.112433 [12864/60000]
loss: 0.062756 [19264/60000]
loss: 0.106322 [25664/60000]
loss: 0.253785 [32064/60000]
loss: 0.098289 [38464/60000]
loss: 0.093422 [44864/60000]
loss: 0.258661 [51264/60000]
loss: 0.246725 [57664/60000]
Test Error:
 Accuracy: 98.5%, Avg loss: 0.044069

Epoch 3
 -------------------------
loss: 0.063777 [   64/60000]
loss: 0.143846 [ 6464/60000]
loss: 0.081036 [12864/60000]
loss: 0.208270 [19264/60000]
loss: 0.020026 [25664/60000]
loss: 0.058829 [32064/60000]
loss: 0.110821 [38464/60000]
loss: 0.1917

In [9]:
torch.save(model.state_dict(), "MNIST_CNN.pth")
print("Saved PyTorch Model State to MNIST_CNN.pth")

Saved PyTorch Model State to MNIST_CNN.pth
